## Test Data Prediction
### Master
This notebook will be used to open test.csv file and use designated model to predict the test data for kaggle submission.

In [29]:
%%bash
git branch

  cost1/feat1/wip
  cost1/feat2/wip
  cost1/wip
  feat1/score1/wip
  feat1/wip
  feat2/score1/wip
  feat2/wip
  feat3/wip
  feat4/wip
  feat5/wip
* master


In [ ]:
%%bash
git add 'Test_data.ipynb'
git commit -m 'Appears that master branch cannot submit test'
git push --set-upstream origin master

In [30]:
model_ver = 'master.sav'

In [31]:
%matplotlib inline 
import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn.model_selection
import pickle

Load data from csv file then prepare the data before inputting into the trained model.

In [32]:
test_ex = pd.read_csv('gender_submission.csv')
test_ex.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [39]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#Concate train and test to avoid mismatch features in the testing phase
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
#dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num].copy()
test = dataset[train_objs_num:].copy()
test.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298


In [41]:
test.isnull().any()

Age                           True
Fare                          True
Parch                        False
SibSp                        False
Survived                      True
Cabin_A11                    False
Cabin_A18                    False
Cabin_A21                    False
Cabin_A29                    False
Cabin_A34                    False
Cabin_A9                     False
Cabin_B10                    False
Cabin_B11                    False
Cabin_B24                    False
Cabin_B26                    False
Cabin_B36                    False
Cabin_B41                    False
Cabin_B45                    False
Cabin_B51 B53 B55            False
Cabin_B52 B54 B56            False
Cabin_B57 B59 B63 B66        False
Cabin_B58 B60                False
Cabin_B61                    False
Cabin_B69                    False
Cabin_B71                    False
Cabin_B78                    False
Cabin_C101                   False
Cabin_C105                   False
Cabin_C106          

In [40]:
test = test.drop('PassengerId',axis=1)
test = 
#test = test.drop('Name',axis=1)

#To flag num attri features
num_attri = ['Age','SibSp','Parch','Fare']
cat_attri = [column for column in test if (column not in num_attri and column != 'Survived')]

#Creating dummy variable
for column in (cat_attri):
    #Lets just keep every categorical attributes for plotting
    tempDF = pd.get_dummies(test[column],prefix=column)
    test.drop(column,axis=1,inplace=True)
    test = test.join(tempDF)
test.head()

,Age,Fare,Parch,SibSp,Survived,Cabin_A11,Cabin_A18,Cabin_A21,Cabin_A29,Cabin_A34,...,Ticket_SOTON/OQ 392083,Ticket_STON/O 2. 3101268,Ticket_STON/O 2. 3101291,Ticket_STON/O2. 3101270,Ticket_STON/OQ. 369943,Ticket_W./C. 14260,Ticket_W./C. 14266,Ticket_W./C. 6607,Ticket_W./C. 6608,Ticket_W.E.P. 5734
0,34.5,7.8292,0,0,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,47.0,7.0000,0,1,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62.0,9.6875,0,0,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,27.0,8.6625,0,0,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22.0,12.2875,1,1,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
loaded_model = pickle.load(open(model_ver, 'rb'))
loaded_model.predict(test)

/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ValueError: feature_names mismatch: [u'Age', u'Fare', u'Parch', u'SibSp', u'Cabin_A10', u'Cabin_A16', u'Cabin_A20', u'Cabin_A23', u'Cabin_A24', u'Cabin_A26', u'Cabin_A31', u'Cabin_A34', u'Cabin_A36', u'Cabin_A5', u'Cabin_A6', u'Cabin_A7', u'Cabin_B101', u'Cabin_B18', u'Cabin_B19', u'Cabin_B20', u'Cabin_B22', u'Cabin_B3', u'Cabin_B30', u'Cabin_B35', u'Cabin_B37', u'Cabin_B38', u'Cabin_B39', u'Cabin_B4', u'Cabin_B41', u'Cabin_B42', u'Cabin_B49', u'Cabin_B5', u'Cabin_B50', u'Cabin_B51 B53 B55', u'Cabin_B57 B59 B63 B66', u'Cabin_B58 B60', u'Cabin_B69', u'Cabin_B71', u'Cabin_B73', u'Cabin_B77', u'Cabin_B79', u'Cabin_B80', u'Cabin_B82 B84', u'Cabin_B86', u'Cabin_B94', u'Cabin_B96 B98', u'Cabin_C101', u'Cabin_C103', u'Cabin_C104', u'Cabin_C110', u'Cabin_C111', u'Cabin_C118', u'Cabin_C123', u'Cabin_C124', u'Cabin_C125', u'Cabin_C126', u'Cabin_C148', u'Cabin_C2', u'Cabin_C22 C26', u'Cabin_C23 C25 C27', u'Cabin_C30', u'Cabin_C32', u'Cabin_C45', u'Cabin_C46', u'Cabin_C49', u'Cabin_C50', u'Cabin_C

In [28]:
# load the model from disk
loaded_model = pickle.load(open(model_ver, 'rb'))


accuracy_score(y_test,clf2.predict(X_test))
result = loaded_model.score(X_test, Y_test)
print(result)

NameError: name 'y_test' is not defined